1. 小区间质押是否可能获得高额利润

In [2]:
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        dailySnapshots(skip: $num_skip, orderBy: day, orderDirection: desc) {
            day
            dailyVolumeUSD
            dailyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

In [3]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))


num_skip = 0
result = []
is_break = False
while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["dailySnapshots"]
    for item in query_result:
        day = int(item["day"])
        if day <= 19477:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)


print(len(result))
print(result[:10])

Querying ticks, num_skip=0
79
[{'day': 19556, 'dailyVolumeUSD': '62645858.87339231038231371303131', 'dailyTotalRevenueUSD': '31322.92943669615519115685651945', 'totalValueLockedUSD': '43991968.86397924299999433932886'}, {'day': 19555, 'dailyVolumeUSD': '78836539.34931248039460595756886', 'dailyTotalRevenueUSD': '39418.2696746562401973029787891', 'totalValueLockedUSD': '44594966.108651452749034656584'}, {'day': 19554, 'dailyVolumeUSD': '43233649.01354901681799617409771', 'dailyTotalRevenueUSD': '21616.82450677450840899808705174', 'totalValueLockedUSD': '46212787.3971710232147209161641'}, {'day': 19553, 'dailyVolumeUSD': '29838778.31630346908133923990871', 'dailyTotalRevenueUSD': '14919.38915815173454066961995703', 'totalValueLockedUSD': '46504482.69780273868477842499168'}, {'day': 19552, 'dailyVolumeUSD': '129294121.16599334024223784584379', 'dailyTotalRevenueUSD': '64647.06058299667012111892292883', 'totalValueLockedUSD': '46430790.59592811944603207193834'}, {'day': 19551, 'dailyVolume

In [4]:
import pandas as pd

from datetime import datetime, timedelta


def unixtime_to_day_before(day_count, unixtime=0):
    current_date = datetime.fromtimestamp(unixtime) + timedelta(days=day_count)
    return current_date.strftime("%Y-%m-%d")


arb_df = pd.DataFrame(result)
arb_df["date"] = arb_df["day"].apply(unixtime_to_day_before)
arb_df.head()

,day,dailyVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD,date
0,19556,62645858.87339231038231371303131,31322.92943669615519115685651945,43991968.86397924299999433932886,2023-07-18
1,19555,78836539.34931248039460595756886,39418.2696746562401973029787891,44594966.108651452749034656584,2023-07-17
2,19554,43233649.01354901681799617409771,21616.82450677450840899808705174,46212787.3971710232147209161641,2023-07-16
3,19553,29838778.31630346908133923990871,14919.38915815173454066961995703,46504482.69780273868477842499168,2023-07-15
4,19552,129294121.16599334024223784584379,64647.06058299667012111892292883,46430790.59592811944603207193834,2023-07-14


In [6]:
arb_df["dailyVolumeUSD"] = arb_df["dailyVolumeUSD"].astype(float).round(2)
arb_df["dailyTotalRevenueUSD"] = arb_df["dailyTotalRevenueUSD"].astype(float).round(2)
arb_df["totalValueLockedUSD"] = arb_df["totalValueLockedUSD"].astype(float).round(2)
arb_df["date"] = arb_df["date"].astype(str)
arb_df = arb_df[['date', 'dailyVolumeUSD', 'dailyTotalRevenueUSD', 'totalValueLockedUSD']]

arb_old_df = pd.read_csv("../../data/arb_data_20230718.csv")
# arb_old_df.rename(columns={'dailyTotalVolumeUSD': 'dailyVolumeUSD'}, inplace=True)
arb_df = pd.concat([arb_df, arb_old_df])
arb_df.to_csv("../../data/arb_data_20230718.csv", index=False)

,date,dailyTotalVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD
0,2023-04-30,1.294347e+08,64717.36,58969074.13
1,2023-04-29,5.831820e+07,29159.10,59522967.00
2,2023-04-28,1.291540e+08,64576.98,58623166.41
3,2023-04-27,2.670841e+08,133542.06,53907496.47
4,2023-04-26,2.884190e+08,144209.50,50551198.16
